## Imports

In [8]:
import sagemaker
import json

## Clients

In [3]:
sagemaker_client = sagemaker.Session().sagemaker_runtime_client

## Constants

In [16]:
OBJECT_DETECTION_ENDPOINT = "object-detection-2023-05-10-18-33-01-848"
THRESH = 0.7

## Prediction

In [23]:
def find_detections(detections):
    found_detections = []
    for detection in detections:
        (klass, conf, x0, y0, x1, y1) = detection
        if conf < THRESH:
            continue
        found_detections.append((klass, conf, (x0, y0), (x1, y1)))
    return found_detections


def predict(img_path):
    data = None
    with open(img_path, "rb") as image:
        data = bytearray(image.read())
    
    if data is None:
        raise Exception(f"image-path={img_path}")
        
    response = sagemaker_client.invoke_endpoint(
        EndpointName=OBJECT_DETECTION_ENDPOINT,
        Body=data,
        ContentType="image/jpeg",
    )
    results = response["Body"].read()
    detections = json.loads(results)
    found_detections = find_detections(detections["prediction"])
    found_detections.sort(key=lambda a, conf, b, c: conf, reverse=True)
    return found_detections


def show_predictions(img_path):
    predictions = predict(img_path)
    if predictions:
        print(f"{img_path} #predictions={len(predictions)}, prediction={predictions[0]}")
    else:
        print(f"{img_path} No predictions")

In [26]:
show_predictions("European_Goldfinch_0003_33066.jpg")

European_Goldfinch_0003_33066.jpg No predictions
